# OS - ELM

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from typing import Optional, Union, List
import warnings
warnings.filterwarnings('ignore')

class RobustOSELM(nn.Module):
    """
    Online Sequential Extreme Learning Machine with micro-batching support.
    
    Features:
    - Micro-batching for computational efficiency
    - Proper hyperparameter ranges
    - Numerical stability improvements
    - Adaptive regularization
    - Better initialization strategies
    """
    
    def __init__(self, 
                 n_hidden: int = 200,
                 activation: str = 'tanh',
                 regularization: float = 0.1,
                 device: str = 'auto',
                 random_state: Optional[int] = None,
                 adaptive_reg: bool = True,
                 micro_batch_size: int = 1):
        super(RobustOSELM, self).__init__()
        
        self.n_hidden = n_hidden
        self.activation = activation
        self.regularization = regularization
        self.adaptive_reg = adaptive_reg
        self.random_state = random_state
        self.micro_batch_size = micro_batch_size
        
        # Set device
        if device == 'auto':
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(device)
        
        # Model parameters
        self.input_weights = None
        self.biases = None
        self.output_weights = None
        self.P_matrix = None  # Precision matrix for online updates
        self.n_features = None
        self.n_classes = None
        self.classes_ = None
        self.is_fitted = False
        self.sample_count = 0
        
        # Set random seed for reproducibility
        if random_state is not None:
            torch.manual_seed(random_state)
            np.random.seed(random_state)
    
    def _get_activation_function(self):
        """Get activation function with improved options."""
        if self.activation == 'sigmoid':
            return lambda x: torch.sigmoid(x)
        elif self.activation == 'tanh':
            return lambda x: torch.tanh(x)
        elif self.activation == 'relu':
            return lambda x: torch.relu(x)
        elif self.activation == 'leaky_relu':
            return lambda x: torch.leaky_relu(x, 0.01)
        elif self.activation == 'swish':
            return lambda x: x * torch.sigmoid(x)
        elif self.activation == 'gelu':
            return lambda x: torch.nn.functional.gelu(x)
        else:
            return lambda x: torch.tanh(x)  # Default fallback
    
    def _initialize_network(self, n_features):
        """Initialize network with proper scaling."""
        self.n_features = n_features
        
        # Xavier/Glorot initialization for better convergence
        if self.activation in ['sigmoid', 'tanh']:
            # Xavier uniform initialization
            limit = np.sqrt(6.0 / (n_features + self.n_hidden))
            self.input_weights = (torch.rand(self.n_hidden, n_features, device=self.device) * 2 * limit) - limit
        else:
            # He initialization for ReLU-like activations
            std = np.sqrt(2.0 / n_features)
            self.input_weights = torch.randn(self.n_hidden, n_features, device=self.device) * std
        
        # Initialize biases
        self.biases = (torch.rand(self.n_hidden, 1, device=self.device) * 2) - 1
    
    def _compute_hidden_layer(self, X):
        """Compute hidden layer activations with numerical stability."""
        if isinstance(X, np.ndarray):
            X = torch.tensor(X, dtype=torch.float32, device=self.device)
        
        # Compute pre-activation: X @ W^T + b
        linear = torch.mm(X, self.input_weights.T) + self.biases.T
        
        # Apply activation with numerical clipping for stability
        activation_fn = self._get_activation_function()
        
        # Clip extreme values to prevent overflow
        linear = torch.clamp(linear, min=-50, max=50)
        hidden = activation_fn(linear)
        
        return hidden
    
    def _encode_targets(self, y):
        """Encode target values appropriately."""
        if self.classes_ is None:
            self.classes_ = np.unique(y)
            self.n_classes = len(self.classes_)
        
        # Create proper encoding
        if self.n_classes == 2:
            # Binary classification: use -1, +1 encoding for better numerical properties
            y_encoded = np.zeros((len(y), 1))
            for i, label in enumerate(y):
                y_encoded[i, 0] = 1.0 if label == self.classes_[1] else -1.0
        else:
            # Multi-class: one-hot encoding
            y_encoded = np.zeros((len(y), self.n_classes))
            for i, label in enumerate(y):
                class_idx = np.where(self.classes_ == label)[0][0]
                y_encoded[i, class_idx] = 1.0
        
        return torch.tensor(y_encoded, dtype=torch.float32, device=self.device)
    
    def _adaptive_regularization(self, H):
        """Compute adaptive regularization based on data characteristics."""
        if not self.adaptive_reg:
            return self.regularization
        
        # Compute condition number estimate
        H_norm = torch.norm(H, p='fro')
        n_samples = H.shape[0]
        
        # Adaptive regularization based on data scale and sample size
        base_reg = self.regularization
        adaptive_factor = max(0.1, min(10.0, H_norm.item() / (n_samples * self.n_hidden)))
        
        return base_reg * adaptive_factor
    
    def initial_training(self, X_init, y_init):
        """Robust initial training phase."""
        X_init = np.array(X_init, dtype=np.float32)
        y_init = np.array(y_init)
        
        n_samples, n_features = X_init.shape
        print(f"Initial training: {n_samples} samples, {n_features} features, {self.n_hidden} neurons")
        
        # Initialize network
        if self.input_weights is None:
            self._initialize_network(n_features)
        
        # Compute hidden layer output
        H = self._compute_hidden_layer(X_init)
        
        # Encode targets
        T = self._encode_targets(y_init)
        
        print(f"Hidden layer shape: {H.shape}, Target shape: {T.shape}")
        
        # Compute output weights with proper regularization
        try:
            # Compute H^T @ H
            HtH = torch.mm(H.T, H)  # (n_hidden, n_hidden)
            
            # Adaptive regularization
            reg_param = self._adaptive_regularization(H)
            reg_matrix = reg_param * torch.eye(self.n_hidden, device=self.device)
            
            # Regularized system: (H^T @ H + λI) @ β = H^T @ T
            HtH_reg = HtH + reg_matrix
            HtT = torch.mm(H.T, T)  # (n_hidden, n_output)
            
            # Solve using Cholesky decomposition for numerical stability
            try:
                L = torch.linalg.cholesky(HtH_reg)
                # Solve L @ y = HtT
                y = torch.linalg.solve_triangular(L, HtT, upper=False)
                # Solve L^T @ β = y
                self.output_weights = torch.linalg.solve_triangular(L.T, y, upper=True)
            except:
                # Fallback to direct solve
                self.output_weights = torch.linalg.solve(HtH_reg, HtT)
            
            # Initialize precision matrix for online updates
            self.P_matrix = torch.linalg.inv(HtH_reg)
            
            print(f"Initial training successful. Regularization: {reg_param:.6f}")
            
        except Exception as e:
            print(f"Initial training failed, using fallback method: {e}")
            # Fallback: use pseudoinverse
            H_pinv = torch.linalg.pinv(H)
            self.output_weights = torch.mm(H_pinv, T)
            self.P_matrix = torch.eye(self.n_hidden, device=self.device) / self.regularization
        
        self.sample_count = n_samples
        self.is_fitted = True
        print(f"Output weights shape: {self.output_weights.shape}")
    
    def sequential_update_microbatch(self, X_batch, y_batch):
        """
        Micro-batch sequential learning update.
        Processes multiple samples together for computational efficiency.
        """
        if not self.is_fitted:
            raise ValueError("Must perform initial training first")
        
        X_batch = np.array(X_batch, dtype=np.float32)
        y_batch = np.array(y_batch)
        batch_size = len(X_batch)
        
        # Compute hidden activations for the entire batch
        H = self._compute_hidden_layer(X_batch)  # (batch_size, n_hidden)
        T = self._encode_targets(y_batch)  # (batch_size, n_output)
        
        try:
            # Batch Sherman-Morrison-Woodbury formula
            # For batch updates, we use the matrix inversion lemma:
            # (A + UCV)^-1 = A^-1 - A^-1 U (C^-1 + V A^-1 U)^-1 V A^-1
            # where A = P_matrix, U = H^T, C = I, V = H
            
            # P @ H^T (n_hidden, batch_size)
            PH_T = torch.mm(self.P_matrix, H.T)
            
            # H @ P @ H^T + I (batch_size, batch_size)
            HPH_T = torch.mm(H, PH_T)
            I_batch = torch.eye(batch_size, device=self.device)
            inv_term = HPH_T + I_batch
            
            # Check for numerical stability
            try:
                inv_HPH_T_I = torch.linalg.inv(inv_term)
            except:
                # Add small regularization for numerical stability
                inv_HPH_T_I = torch.linalg.inv(inv_term + 1e-8 * I_batch)
            
            # Update precision matrix using Woodbury identity
            # P_new = P - P @ H^T @ (H @ P @ H^T + I)^-1 @ H @ P
            temp = torch.mm(PH_T, inv_HPH_T_I)
            P_update = torch.mm(temp, torch.mm(H, self.P_matrix))
            self.P_matrix = self.P_matrix - P_update
            
            # Compute prediction errors for the batch
            current_predictions = torch.mm(H, self.output_weights)  # (batch_size, n_output)
            errors = T - current_predictions  # (batch_size, n_output)
            
            # Update output weights
            # β_new = β + P @ H^T @ (H @ P @ H^T + I)^-1 @ errors
            weight_update = torch.mm(temp, errors)  # (n_hidden, n_output)
            self.output_weights = self.output_weights + weight_update
            
            self.sample_count += batch_size
            
        except Exception as e:
            print(f"Micro-batch update warning: {e}")
            # Fallback to individual updates
            for i in range(batch_size):
                try:
                    self.sequential_update_single(X_batch[i:i+1], y_batch[i:i+1])
                except:
                    continue
    
    def sequential_update_single(self, X_new, y_new):
        """Single sample sequential learning update (fallback method)."""
        if not self.is_fitted:
            raise ValueError("Must perform initial training first")
        
        X_new = np.array(X_new, dtype=np.float32)
        y_new = np.array(y_new)
        
        # Compute hidden activation for this sample
        h = self._compute_hidden_layer(X_new)  # (1, n_hidden)
        h_T = h.T  # (n_hidden, 1)
        
        # Encode target
        t = self._encode_targets(y_new)  # (1, n_output)
        t_T = t.T  # (n_output, 1)
        
        try:
            # Sherman-Morrison formula with numerical safeguards
            Ph = torch.mm(self.P_matrix, h_T)  # (n_hidden, 1)
            hPh = torch.mm(h, Ph)  # (1, 1)
            denominator = 1.0 + hPh.item()
            
            # Numerical stability check
            if abs(denominator) > 1e-8:
                # Update precision matrix
                P_update = torch.mm(Ph, Ph.T) / denominator
                self.P_matrix = self.P_matrix - P_update
                
                # Compute prediction error
                prediction = torch.mm(self.output_weights.T, h_T)  # (n_output, 1)
                error = t_T - prediction
                
                # Update output weights
                weight_update = torch.mm(Ph, error.T) / denominator  # (n_hidden, n_output)
                self.output_weights = self.output_weights + weight_update
                
                self.sample_count += 1
                
        except Exception as e:
            print(f"Single update warning: {e}")
    
    def sequential_update(self, X_new, y_new):
        """
        Enhanced sequential update with micro-batching support.
        Automatically handles batching based on micro_batch_size.
        """
        X_new = np.array(X_new, dtype=np.float32)
        y_new = np.array(y_new)
        n_samples = len(X_new)
        
        # Process in micro-batches
        if self.micro_batch_size > 1:
            n_batches = (n_samples + self.micro_batch_size - 1) // self.micro_batch_size
            
            for batch_idx in range(n_batches):
                start_idx = batch_idx * self.micro_batch_size
                end_idx = min(start_idx + self.micro_batch_size, n_samples)
                
                X_batch = X_new[start_idx:end_idx]
                y_batch = y_new[start_idx:end_idx]
                
                self.sequential_update_microbatch(X_batch, y_batch)
        else:
            # Original single-sample processing
            for i in range(n_samples):
                self.sequential_update_single(X_new[i:i+1], y_new[i:i+1])
    
    def fit(self, X, y, initial_ratio: float = 0.3):
        """
        Fit the model with optimal initial/sequential split and micro-batching.
        
        Args:
            X: Training features
            y: Training labels
            initial_ratio: Fraction of data for initial training (0.2-0.5 recommended)
        """
        X = np.array(X, dtype=np.float32)
        y = np.array(y)
        
        n_samples = len(X)
        n_initial = max(self.n_hidden, int(n_samples * initial_ratio))
        n_initial = min(n_initial, n_samples)
        
        print(f"Training strategy: {n_initial} initial, {n_samples - n_initial} sequential")
        print(f"Micro-batch size: {self.micro_batch_size}")
        
        # Initial training
        self.initial_training(X[:n_initial], y[:n_initial])
        
        # Sequential training with micro-batching
        if n_samples > n_initial:
            remaining_X = X[n_initial:]
            remaining_y = y[n_initial:]
            
            if self.micro_batch_size > 1:
                n_microbatches = (len(remaining_X) + self.micro_batch_size - 1) // self.micro_batch_size
                print(f"Sequential training: {n_microbatches} micro-batches of size ≤{self.micro_batch_size}")
            else:
                print(f"Sequential training: {len(remaining_X)} individual samples")
            
            self.sequential_update(remaining_X, remaining_y)
        
        return self
    
    def predict_proba(self, X):
        """Predict class probabilities."""
        if not self.is_fitted:
            raise ValueError("Model not fitted yet")
        
        # Compute hidden layer activations
        H = self._compute_hidden_layer(X)
        
        # Compute raw predictions
        raw_output = torch.mm(H, self.output_weights)  
        
        if self.n_classes == 2:
            # Binary classification with sigmoid
            probs = torch.sigmoid(raw_output).cpu().numpy()
            # Return probabilities for both classes
            return np.column_stack([1 - probs, probs])
        else:
            # Multi-class with softmax
            probs = torch.softmax(raw_output, dim=1).cpu().numpy()
            return probs
    
    def predict(self, X):
        """Predict class labels."""
        probs = self.predict_proba(X)
        predicted_indices = np.argmax(probs, axis=1)
        return self.classes_[predicted_indices]
    
    def score(self, X, y):
        """Compute accuracy score."""
        return accuracy_score(y, self.predict(X))

def comprehensive_elm_tuning(file_path, test_configurations, device='auto'):
    """
    Comprehensive ELM tuning with micro-batching support.
    """
    filename = os.path.basename(file_path)
    print(f"\n{'='*70}")
    print(f"COMPREHENSIVE OS-ELM TUNING FOR {filename}")
    print(f"{'='*70}")

    # Load and preprocess data
    try:
        df = pd.read_csv(file_path)
        print(f"Dataset loaded: {df.shape}")
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        return []

    # Data preprocessing
    if df.isnull().sum().sum() > 0:
        print("Handling missing values...")
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())
    
    # Prepare features and targets
    X = df.iloc[:, :-1].values.astype(np.float32)
    y = df.iloc[:, -1].values
    
    print(f"Features: {X.shape[1]}, Samples: {X.shape[0]}")
    print(f"Classes: {np.unique(y)}, Distribution: {np.bincount(y.astype(int))}")
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Training samples: {len(X_train_scaled)}, Test samples: {len(X_test_scaled)}")
    
    # Run experiments
    results = []
    total_configs = len(test_configurations)
    
    for config_idx, config in enumerate(test_configurations):
        print(f"\n{'='*50}")
        print(f"Configuration {config_idx + 1}/{total_configs}")
        print(f"Config: {config}")
        print(f"{'='*50}")
        
        try:
            # Create model
            model = RobustOSELM(
                n_hidden=config['n_hidden'],
                activation=config['activation'],
                regularization=config['regularization'],
                device=device,
                random_state=42,
                adaptive_reg=config.get('adaptive_reg', True),
                micro_batch_size=config.get('micro_batch_size', 1)
            )
            
            # Training
            start_time = time.time()
            model.fit(X_train_scaled, y_train, initial_ratio=config.get('initial_ratio', 0.3))
            training_time = time.time() - start_time
            
            # Evaluation
            train_accuracy = model.score(X_train_scaled, y_train)
            test_accuracy = model.score(X_test_scaled, y_test)
            
            result = {
                'n_hidden': config['n_hidden'],
                'activation': config['activation'],
                'regularization': config['regularization'],
                'initial_ratio': config.get('initial_ratio', 0.3),
                'adaptive_reg': config.get('adaptive_reg', True),
                'micro_batch_size': config.get('micro_batch_size', 1),
                'train_accuracy': train_accuracy,
                'test_accuracy': test_accuracy,
                'training_time': training_time,
                'overfitting': train_accuracy - test_accuracy
            }
            
            results.append(result)
            
            print(f"Results:")
            print(f"  Train Accuracy: {train_accuracy*100:.2f}%")
            print(f"  Test Accuracy:  {test_accuracy*100:.2f}%")
            print(f"  Training Time:  {training_time:.3f}s")
            print(f"  Overfitting:    {(train_accuracy - test_accuracy)*100:.2f}%")
            
            
            if device != 'cpu':
                torch.cuda.empty_cache()
                
        except Exception as e:
            print(f"Configuration failed: {e}")
            results.append({
                'n_hidden': config['n_hidden'],
                'activation': config['activation'],
                'regularization': config['regularization'],
                'initial_ratio': config.get('initial_ratio', 0.3),
                'adaptive_reg': config.get('adaptive_reg', True),
                'micro_batch_size': config.get('micro_batch_size', 1),
                'train_accuracy': 0.0,
                'test_accuracy': 0.0,
                'training_time': float('inf'),
                'overfitting': float('inf')
            })
    
    # Analysis and visualization
    analyze_results(results, filename)
    
    return results

def analyze_results(results, filename):
    """Comprehensive analysis of tuning results."""
    if not results:
        print("No results to analyze")
        return
    
    df_results = pd.DataFrame(results)
    valid_results = df_results[df_results['test_accuracy'] > 0].copy()
    
    if valid_results.empty:
        print("No valid results found")
        return
    
    print(f"\n{'='*50}")
    print("ANALYSIS SUMMARY")
    print(f"{'='*50}")
    
    # Best configuration
    best_idx = valid_results['test_accuracy'].idxmax()
    best_config = valid_results.iloc[best_idx]
    
    print(f"Best Configuration:")
    print(f"  Hidden Neurons: {best_config['n_hidden']}")
    print(f"  Activation: {best_config['activation']}")
    print(f"  Regularization: {best_config['regularization']:.4f}")
    print(f"  Micro-batch Size: {best_config['micro_batch_size']}")
    print(f"  Test Accuracy: {best_config['test_accuracy']*100:.2f}%")
    print(f"  Training Time: {best_config['training_time']:.3f}s")
    
    
    if 'micro_batch_size' in valid_results.columns:
        print(f"\nPerformance by Micro-batch Size:")
        for batch_size in sorted(valid_results['micro_batch_size'].unique()):
            subset = valid_results[valid_results['micro_batch_size'] == batch_size]
            avg_acc = subset['test_accuracy'].mean()
            avg_time = subset['training_time'].mean()
            print(f"  Batch size {batch_size}: {avg_acc*100:.2f}% accuracy, {avg_time:.3f}s avg time")
    
    # Statistics by activation function
    print(f"\nPerformance by Activation Function:")
    for activation in valid_results['activation'].unique():
        subset = valid_results[valid_results['activation'] == activation]
        avg_acc = subset['test_accuracy'].mean()
        std_acc = subset['test_accuracy'].std()
        print(f"  {activation}: {avg_acc*100:.2f}% ± {std_acc*100:.2f}%")

# Example usage with micro-batching configurations
if __name__ == "__main__":
    # Enhanced configurations with micro-batching
    test_configurations = [
        # Compare micro-batch sizes with moderate hidden neurons
        {'n_hidden': 2500, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 1},
        {'n_hidden': 2500, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 5},
        {'n_hidden': 2500, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 10},
        {'n_hidden': 2500, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 20},
        
        # Test larger hidden neurons with micro-batching
        {'n_hidden': 5000, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 10},
        {'n_hidden': 7500, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 10},
        {'n_hidden': 10000, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 10},
        
        # Compare activations with micro-batching
        {'n_hidden': 5000, 'activation': 'tanh', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 10},
        {'n_hidden': 5000, 'activation': 'relu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 10},
        {'n_hidden': 5000, 'activation': 'swish', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 10},
    ]
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    DATASET_PATH = '../Datasets'
    print(f"Using device: {device}")
    
    if os.path.exists(DATASET_PATH):
        dataset_files = [os.path.join(DATASET_PATH, f) 
                        for f in os.listdir(DATASET_PATH) 
                        if f.endswith('.csv')]
        
        print(f"Found {len(dataset_files)} dataset files")
        
        all_results = {}
        for file in dataset_files:
            results = comprehensive_elm_tuning(file, test_configurations, device)
            all_results[file] = results
        
        # Overall summary
        print(f"\n{'='*70}")
        print("OVERALL SUMMARY ACROSS ALL DATASETS")
        print(f"{'='*70}")
        
        for file, results in all_results.items():
            if results:
                df_res = pd.DataFrame(results)
                valid_res = df_res[df_res['test_accuracy'] > 0]
                if not valid_res.empty:
                    best_acc = valid_res['test_accuracy'].max()
                    filename = os.path.basename(file)
                    print(f"{filename}: Best accuracy = {best_acc*100:.2f}%")
        
    else:
        print(f"Dataset path {DATASET_PATH} not found!")
    
    print("\nComprehensive tuning completed!")

Using device: cuda
Found 3 dataset files

COMPREHENSIVE OS-ELM TUNING FOR MEFAR_DOWN.csv
Dataset loaded: (27570, 18)
Handling missing values...
Features: 17, Samples: 27570
Classes: [0. 1.], Distribution: [13106 14464]
Training samples: 22056, Test samples: 5514

Configuration 1/10
Config: {'n_hidden': 2500, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 1}
Training strategy: 2500 initial, 19556 sequential
Micro-batch size: 1
Initial training: 2500 samples, 17 features, 2500 neurons
Hidden layer shape: torch.Size([2500, 2500]), Target shape: torch.Size([2500, 1])
Initial training successful. Regularization: 0.010000
Output weights shape: torch.Size([2500, 1])
Sequential training: 19556 individual samples
Results:
  Train Accuracy: 47.39%
  Test Accuracy:  47.61%
  Training Time:  19.795s
  Overfitting:    -0.22%

Configuration 2/10
Config: {'n_hidden': 2500, 'activation': 'gelu', 'regularization': 0.1, 'initial_ratio': 0.1, 'micro_batch_size': 5}

KeyboardInterrupt: 